In [1]:
#=======================================================================================
# sentence-bert와 Faiss 라이브러리를 이용하여 search 하는 예시임
# => Faiss는 facebook에서 만든 vector 유사도를 측정하는 라이브러리로, 
# 기존 numpy 나 scikit-learn 에서 제공하는 cosine similarity 보다 강력하며, GPU도 지원한다.
#
# 참고 
# 번역본 : https://ichi.pro/ko/transformers-mich-faissleul-sayonghayeo-simaentig-geomsaeg-enjin-eul-guchughaneun-bangbeob-242711337083112
# (원본 : https://medium.com/towards-data-science/how-to-build-a-semantic-search-engine-with-transformers-and-faiss-dcbea307a0e8)
#
# 설치
# Faiss는 설치는 https://github.com/facebookresearch/faiss/blob/main/INSTALL.md 참조
# GPU 버전 : $ conda install -c conda-forge faiss-gpu
# 
#=======================================================================================

import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('..')
from myutils import GPU_info, seed_everything

device = GPU_info()
seed_everything(111)

logfilepath:bwdataset_2022-03-28.log
logfilepath:qnadataset_2022-03-28.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
# sentence bert 로딩
from sentence_transformers import SentenceTransformer
smodel_path = "../model/bert/bmc_fpt_kowiki20200920.train-sbert-tsmodel-0310"
smodel = SentenceTransformer(smodel_path)
smodel.to(device)


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [3]:
# list 데이터들을 dataframe으로 만듬
data = [
        [1, '오늘 하루는 날씨가 흐리고 비가 온다'], 
        [11,'내일은 오늘보다 더춥고, 가끔 눈이 오겠다고 한다'],
        [21,'오늘은 가족들과 함께 등산을 하기로 했다'],
        [2,'점점 날씨가 따뜻해 지면서 봄이 오는걸 느낄수 있다'],
        [3,'오늘부터 다음주 까지는 재택근무이다'],
        [31,'오늘 첫눈이 올거라고 한다'],
        [4,'오늘 기온이 올라 화창한 봄날씨가 될거라고 한다']
       ]
df = pd.DataFrame(data, columns=['uid','text'])
print(df['text'].values)

['오늘 하루는 날씨가 흐리고 비가 온다' '내일은 오늘보다 더춥고, 가끔 눈이 오겠다고 한다'
 '오늘은 가족들과 함께 등산을 하기로 했다' '점점 날씨가 따뜻해 지면서 봄이 오는걸 느낄수 있다'
 '오늘부터 다음주 까지는 재택근무이다' '오늘 첫눈이 올거라고 한다' '오늘 기온이 올라 화창한 봄날씨가 될거라고 한다']


In [18]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [20]:
def embeddingforfaiss(df):
    # embedding 생성(인코딩)

    start = time.time()
    embeddings = smodel.encode(df.text.to_list(), show_progress_bar=True, convert_to_tensor=False)

    #float32 로 embeddings 타입 변경
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

    # instance index 생성
    index = faiss.IndexFlatL2(embeddings.shape[1])

    # id를 매핑 시켜줌
    index = faiss.IndexIDMap(index)
    index.add_with_ids(embeddings, df.uid.values)

    print(f'인코딩 시간 : {time.time()-start:.4f}')

In [21]:
#dataframe 인코딩
embeddingforfaiss(df)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

인코딩 시간 : 0.0743


In [22]:
# 여러문장 쿼리 해봄
user_query = ["나는 오늘 여행갈거라서 너무 행복하다", 
             "나는 오늘 너무 슬프다"]

vector = smodel.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=4)

#print(distance)
#print(idx)
#print([list(df[df.idx == num]['text']) for num in idx[0]])
#print([list(df[df.idx == num]['text']) for num in idx[1]])

for i, query in enumerate(user_query):
    print(f'Q: {query}')
    count = 0
    for num in idx[i]:
        text = df[df.uid == num]['text'].values
        print(f'{num}, {text[0]}({distance[i][count]:.3f})')
        count += 1
    print('\n')
        


Q: 나는 오늘 여행갈거라서 너무 행복하다
5, 정말 감동적인 영화를 보았다(189.405)
61, 레스토랑에서 맛있게 점심을 먹었다(275.348)
7, 6시 이후에는 퇴근 길이 너무 막힌다(277.269)
71, 오늘 손홍민이 하는 국가대표 축구 경기를 봤다(279.771)


Q: 나는 오늘 너무 슬프다
7, 6시 이후에는 퇴근 길이 너무 막힌다(266.016)
6, 회사가는 출근길이 너무 막힌다(280.689)
8, tv에서 드라마를 보면서 울었다(301.415)
5, 정말 감동적인 영화를 보았다(364.928)




In [9]:
# 데이터 추가
# list 데이터들을 dataframe으로 만듬
data1 = [
        [5, '정말 감동적인 영화를 보았다'], 
        [51,'드디어 오늘 학교 간다'],
        [61,'레스토랑에서 맛있게 점심을 먹었다'],
        [6,'회사가는 출근길이 너무 막힌다'],
        [7,'6시 이후에는 퇴근 길이 너무 막힌다'],
        [71,'오늘 손홍민이 하는 국가대표 축구 경기를 봤다'],
        [8,'tv에서 드라마를 보면서 울었다']
       ]
df1 = pd.DataFrame(data1, columns=['uid','text'])
print(df1['text'].values)

['정말 감동적인 영화를 보았다' '드디어 오늘 학교 간다' '레스토랑에서 맛있게 점심을 먹었다' '회사가는 출근길이 너무 막힌다'
 '6시 이후에는 퇴근 길이 너무 막힌다' '오늘 손홍민이 하는 국가대표 축구 경기를 봤다' 'tv에서 드라마를 보면서 울었다']


In [23]:
# 추가된 문장 embedding 생성(인코딩)
embeddingforfaiss(df1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

인코딩 시간 : 0.0699


In [24]:
# 추가된 dataframe을 기존 df에 추가함
df = pd.concat([df, df1])
print(df)

   uid                          text
0    1          오늘 하루는 날씨가 흐리고 비가 온다
1   11   내일은 오늘보다 더춥고, 가끔 눈이 오겠다고 한다
2   21        오늘은 가족들과 함께 등산을 하기로 했다
3    2  점점 날씨가 따뜻해 지면서 봄이 오는걸 느낄수 있다
4    3           오늘부터 다음주 까지는 재택근무이다
5   31                오늘 첫눈이 올거라고 한다
6    4    오늘 기온이 올라 화창한 봄날씨가 될거라고 한다
0    5               정말 감동적인 영화를 보았다
1   51                  드디어 오늘 학교 간다
2   61            레스토랑에서 맛있게 점심을 먹었다
3    6              회사가는 출근길이 너무 막힌다
4    7          6시 이후에는 퇴근 길이 너무 막힌다
5   71     오늘 손홍민이 하는 국가대표 축구 경기를 봤다
6    8             tv에서 드라마를 보면서 울었다
0    5               정말 감동적인 영화를 보았다
1   51                  드디어 오늘 학교 간다
2   61            레스토랑에서 맛있게 점심을 먹었다
3    6              회사가는 출근길이 너무 막힌다
4    7          6시 이후에는 퇴근 길이 너무 막힌다
5   71     오늘 손홍민이 하는 국가대표 축구 경기를 봤다
6    8             tv에서 드라마를 보면서 울었다


In [27]:
# 여러문장 쿼리 해봄
user_query = ["나는 오늘 여행갈거라서 너무 행복하다", 
             "나는 오늘 너무 슬프다"]

vector = smodel.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=4)

#print(distance)
#print(idx)
#print([list(df[df.uid == num]['text']) for num in idx[0]])
#print([list(df[df.uid == num]['text']) for num in idx[1]])


for i, query in enumerate(user_query):
    print(f'Q: {query}')
    count = 0
    for num in idx[i]:
        text = df[df.uid == num]['text'].values
        print(f'{num}, {text[0]}({distance[i][count]:.3f})')
        count += 1
    print('\n')
     

Q: 나는 오늘 여행갈거라서 너무 행복하다
5, 정말 감동적인 영화를 보았다(189.405)
61, 레스토랑에서 맛있게 점심을 먹었다(275.348)
7, 6시 이후에는 퇴근 길이 너무 막힌다(277.269)
71, 오늘 손홍민이 하는 국가대표 축구 경기를 봤다(279.771)


Q: 나는 오늘 너무 슬프다
7, 6시 이후에는 퇴근 길이 너무 막힌다(266.016)
6, 회사가는 출근길이 너무 막힌다(280.689)
8, tv에서 드라마를 보면서 울었다(301.415)
5, 정말 감동적인 영화를 보았다(364.928)


